In [81]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os
from torchvision.io import read_image
from PIL import Image



niftypath = "/mnt/c/Users/Patrick/Documents/MPHYS_DATA_NIFTY"
croppath = "/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED"

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [10]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [96]:
idx = 0
transform = ToTensor()

class CustomImageDataset():
    def __init__(self, croppath, transform=None, target_transfrom=None):
        self.img_labels = os.listdir(croppath)
        self.img_dir = croppath
        self.transform=transform
        #print(self.transform)
        self.target_transform=target_transfrom

    def __len__ (self):
        return len(self.img_labels)

    def __get_item__ (self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx])
        print(img_path)
        image = sitk.ReadImage(img_path)
        array = sitk.GetArrayFromImage(image)
        tensor = torch.from_numpy(array)
        #pil_img = Image.fromarray(array)
        label = self.img_labels[idx]
        # if self.transform(array):
        #     array = ToTensor(array)
        # if self.target_transform(label):
        #     label = self.target_transform(label)
        return(tensor,label)
    

for i in range(len(os.listdir(croppath))):
    dataset = CustomImageDataset(croppath, ToTensor(), None)
    array, label = dataset.__get_item__(i)
    print(array.dtype)
    
    
    



/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-001-CT.nii
torch.int32
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-001-RTSTRUCT.nii
torch.float32
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-002-CT.nii
torch.int16
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-002-RTSTRUCT.nii
torch.float32
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-CT.nii
torch.int16
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-RTSTRUCT.nii
torch.float32
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-004-CT.nii
torch.int32
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-004-RTSTRUCT.nii
torch.float32
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-005-CT.nii
torch.int16
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-005-RTSTRUCT.nii
torch.float32
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-006-CT.nii
torch.int16
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-006-RTSTRUCT.nii
torch.float32
/mnt